#### Libraries

In [129]:
import numpy as np
import pandas as pd
import math
import random


In [130]:
df = pd.read_csv('Data.csv')
df.head()

,Val,Id
0,2.586494,2
1,0.932478,2
2,2.361481,2
3,2.042753,2
4,4.973268,3


In [133]:
# k =number of clusters
k =3
# intial_params contains all the intial parameters for the Guassians
initial_params = pd.DataFrame()
#initalizing all means to the same value
random_error = random.sample(range(0, 60), k)
initial_params['means'] = pd.Series([df['Val'].mean() for i in range(k)])
#initals all stds to the same std
initial_params['std'] = pd.Series([df['Val'].std() for i in range (k)])
#intialialize all weights to be equal
initial_weights =[0.1,0.6,0.3]
initial_params['weights'] = pd.Series(initial_weights)
initial_params


,means,std,weights
0,2.04191,2.26236,0.1
1,2.04191,2.26236,0.6
2,2.04191,2.26236,0.3


In [134]:

from scipy.stats import norm

def Guassian_prob(x,mean,std):
    #returns Pr(x<Val) if x is normally distributed
    return norm.pdf(x,mean, std)

def Expectation_Maximization(df, initial_params,k, max_iterations):
    for t in range(max_iterations):
        
        if(t%10==0):
            print("this is ",t," iteration")
            display(initial_params)
        #Expectation_Step

        #name is just so that the dataframe has an appropriate name scheme
        name = 'Gamma_'
        #Gammas contains all probs
        Gammas = pd.DataFrame()
        temp_list =[]
        for i in range(k):
            #Iterate over number of Clusters
            for vals in df['Val']:
                #Iterate over all vals
                temp_list.append(initial_params['weights'][i]*Guassian_prob(vals,initial_params['means'][i],initial_params['std'][i]))
            #compute likelihoods and store in Gamma
            Gammas[name+str(i+1)] =pd.Series(temp_list)
            temp_list=[]

        #Normalizes all columns
        Gammas= Gammas.div(Gammas.sum(axis=1), axis=0)

        #Maximization Step



        pi_k = []
        mu_k = []
        std_k =[]
        #Sum=[]
        for columns in Gammas.columns:
            #total equals total values in dataset
            total =Gammas.count()[0]
            #Sum is the row wise sum of all probs
            Sum = Gammas[columns].sum(axis=0)
            #Using formula for pi_k
           
            pi_k.append(Sum/total)
            #multplying value and prob together
            temp = [val*prob for val,prob in zip(df['Val'],Gammas[columns])]
            #using value of Means
            mu_k.append(sum(temp)/Sum)
        #display(mu_k)
        i =0
        #seperate loop for stds
        for columns in Gammas.columns:
            temp = [prob*(val-mu_k[i])**2 for val,prob in zip(df['Val'],Gammas[columns])]
            Sum = Gammas[columns].sum(axis=0)
            #display(sum(temp))
            #Formula for stds_k
            std_k.append(sum(temp)/Sum)
            i+=1

        initial_params['weights'] =pd.Series(pi_k)
        initial_params['means'] =pd.Series(mu_k)
        initial_params['std'] =pd.Series(std_k)
        Gammas= pd.DataFrame()

    
        
        
    
            

        
       
        
        
    
    
    return initial_params
    
    
l = Expectation_Maximization(df,initial_params,k,20)
l

this is  0  iteration


,means,std,weights
0,2.04191,2.26236,0.1
1,2.04191,2.26236,0.6
2,2.04191,2.26236,0.3


this is  10  iteration


,means,std,weights
0,2.04191,5.067089,0.1
1,2.04191,5.067089,0.6
2,2.04191,5.067089,0.3


,means,std,weights
0,2.04191,5.067089,0.1
1,2.04191,5.067089,0.6
2,2.04191,5.067089,0.3
